In [1]:
import torch
import torch.nn as nn
import torch.optim as optim #优化
import torch.nn.functional as F #激活函数
import numpy as np
from post_clustering import spectral_clustering, acc, nmi #谱聚类
import scipy.io as sio
import math

class Conv2dSamePad(nn.Module):
    
    # 实现tensorflow中same的填充方式
    # 在卷积操作后保持输入和输出的空间尽可能相同
    # self代指类的实例本身
    def __init__(self, kernel_size, stride):#卷积核的大小，步幅的大小
        super(Conv2dSamePad, self).__init__()
        self.kernel_size = kernel_size if type(kernel_size) in [list, tuple] else [kernel_size, kernel_size]
        self.stride = stride if type(stride) in [list, tuple] else [stride, stride]
    #计算输入张量通过卷积层后的输出张量
    def forward(self, x):
        in_height = x.size(2)#输入的张量高度
        in_width = x.size(3)#宽度
        out_height = math.ceil(float(in_height) / float(self.stride[0]))
        #经过卷积这个操作后输出的高度
        out_width = math.ceil(float(in_width) / float(self.stride[1]))
        #卷积后输出的的宽度
        #out_height 和 out_width: 计算经过卷积操作后，输出的张量高度和宽度。
        #math.ceil 函数用于向上取整，
        #这是因为卷积的步幅可能无法整除输入的维度，导致输出维度为小数。
        pad_along_height = ((out_height - 1) * self.stride[0] + self.kernel_size[0] - in_height)
        pad_along_width = ((out_width - 1) * self.stride[1] + self.kernel_size[1] - in_width)
        #pad_along_height 和 pad_along_width: 计算在高度和宽度方向上需要填充的像素数。
        #这是为了确保输出的尺寸与输入的尺寸尽可能相同。
        pad_top = math.floor(pad_along_height / 2)
        pad_left = math.floor(pad_along_width / 2)
        pad_bottom = pad_along_height - pad_top
        pad_right = pad_along_width - pad_left
        #pad_top 和 pad_left: 在高度和宽度方向上的上方和左侧需要填充的像素数。
        #math.floor 用于向下取整。
        #pad_bottom 和 pad_right: 在高度和宽度方向上的下方和右侧需要填充的像素数，通过减法计算得到。
        return F.pad(x, [pad_left, pad_right, pad_top, pad_bottom], 'constant', 0)
        #F.pad: 使用 PyTorch 的 pad 函数对输入张量 x 进行填充，按照计算得到的填充量 [pad_left, pad_right, pad_top, pad_bottom] 
        #在四个方向上进行填充。填充值为 0（即填充的区域为黑色或空白）
        #填充顺序不能变，传参顺序
class ConvTranspose2dSamePad(nn.Module):
    # 对反卷积（输出尺寸大于目标尺寸的情况）操作的结果进行裁剪的过程
    def __init__(self, kernel_size, stride):
        super(ConvTranspose2dSamePad, self).__init__()
        self.kernel_size = kernel_size if type(kernel_size) in [list, tuple] else [kernel_size, kernel_size]
        self.stride = stride if type(stride) in [list, tuple] else [stride, stride]

    def forward(self, x):
        in_height = x.size(2)
        in_width = x.size(3)
        pad_height = self.kernel_size[0] - self.stride[0]
        pad_width = self.kernel_size[1] - self.stride[1]
        #高度和宽度上需要裁剪的像素数
        pad_top = pad_height // 2
        pad_bottom = pad_height - pad_top
        pad_left = pad_width // 2
        pad_right = pad_width - pad_left
        return x[:, :, pad_top:in_height - pad_bottom, pad_left: in_width - pad_right]
 # 在图片长度和宽度一样，同时卷积核的长度和宽度一样的情况下填充没有什么错误
 
# 卷积自编码器：目的是实现特征提取和数据压缩
class ConvAE(nn.Module):
    def __init__(self, channels, kernels):
        """
        channels列表：包含每一层的通道数，灰度：1,RGB:3
        大小，它的长度应该比channels少一个，因为每一层卷积都需要一个卷积核。
    
        """
        super(ConvAE, self).__init__()
        assert isinstance(channels, list) and isinstance(kernels, list)
        #这里是一个断言调试，表示进入后续的一定要是一个list类型的变量
        self.encoder = nn.Sequential()
        #是 PyTorch 提供的一个容器类，我们这里实现的是一个编码器的过程
        #它允许你将多个神经网络层（如卷积层、激活函数等）按照它们被添加的顺序组合在一起。
        for i in range(1, len(channels)):
            #  Each layer will divide the size of feature map by 2
            #  步幅为2，特征（新生成的图像尺寸）通常会减半
            self.encoder.add_module('pad%d' % i, Conv2dSamePad(kernels[i - 1], 2))
            self.encoder.add_module('conv%d' % i,
                                    nn.Conv2d(channels[i - 1], channels[i], kernel_size=kernels[i - 1], stride=2))
            # 卷积层数，输入卷积层的通道数，输出到下一层的卷积通道数，卷积核的大小，步幅的大小
            # 
            self.encoder.add_module('relu%d' % i, nn.ReLU(True))
        #解码器
        self.decoder = nn.Sequential()
        channels = list(reversed(channels))
        kernels = list(reversed(kernels))
        #将通道和卷积核的列表反转，因为解码器的结构与编码器相反。
        for i in range(len(channels) - 1):
            # Each layer will double the size of feature map
            self.decoder.add_module('deconv%d' % (i + 1),
                                    nn.ConvTranspose2d(channels[i], channels[i + 1], kernel_size=kernels[i], stride=2))
            self.decoder.add_module('padd%d' % i, ConvTranspose2dSamePad(kernels[i], 2))
            self.decoder.add_module('relud%d' % i, nn.ReLU(True))
#填充的目的->边缘可处理->
    def forward(self, x):
        h = self.encoder(x)
        y = self.decoder(h)
        return y


class SelfExpression(nn.Module):
    def __init__(self, n):
        super(SelfExpression, self).__init__()
        self.Coefficient = nn.Parameter(1.0e-8 * torch.ones(n, n, dtype=torch.float32), requires_grad=True)
    def forward(self, x):  # shape=[n, d]
        y = torch.matmul(self.Coefficient, x)
        return y


class DSCNet(nn.Module):
    def __init__(self, channels, kernels, num_sample):
        super(DSCNet, self).__init__()
        self.n = num_sample
        self.ae = ConvAE(channels, kernels)
        self.self_expression = SelfExpression(self.n)

    def forward(self, x):  # shape=[n, c, w, h]
        z = self.ae.encoder(x)

        # self expression layer, reshape to vectors, multiply Coefficient, then reshape back
        shape = z.shape
        z = z.view(self.n, -1)  # shape=[n, d]
        z_recon = self.self_expression(z)  # shape=[n, d]
        z_recon_reshape = z_recon.view(shape)

        x_recon = self.ae.decoder(z_recon_reshape)  # shape=[n, c, w, h]
        return x_recon, z, z_recon

    def loss_fn(self, x, x_recon, z, z_recon, weight_coef, weight_selfExp):
        loss_ae = F.mse_loss(x_recon, x, reduction='sum')
        loss_coef = torch.sum(torch.pow(self.self_expression.Coefficient, 2))
        loss_selfExp = F.mse_loss(z_recon, z, reduction='sum')
        loss = loss_ae + weight_coef * loss_coef + weight_selfExp * loss_selfExp

        return loss
    
    '''
    自编码损失 (loss_ae)：衡量重构输入与原始输入之间的差异。
    系数矩阵的 L2 正则化损失 (loss_coef)：用于约束自表达系数矩阵，防止过拟合。
    自表达损失 (loss_selfExp)：衡量通过自表达层处理后的特征与原始特征之间的差异。
    '''


def train(model,  # type: DSCNet
          x, y, epochs, lr=1e-3, weight_coef=1.0, weight_selfExp=150, device='cuda',
          alpha=0.04, dim_subspace=12, ro=8, show=10):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, dtype=torch.float32, device=device)
    x = x.to(device)
    if isinstance(y, torch.Tensor):
        y = y.to('cpu').numpy()
    K = len(np.unique(y))
    for epoch in range(epochs):
        x_recon, z, z_recon = model(x)
        loss = model.loss_fn(x, x_recon, z, z_recon, weight_coef=weight_coef, weight_selfExp=weight_selfExp)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % show == 0 or epoch == epochs - 1:
            C = model.self_expression.Coefficient.detach().to('cpu').numpy()
            y_pred = spectral_clustering(C, K, dim_subspace, alpha, ro)
            print('Epoch %02d: loss=%.4f, acc=%.4f, nmi=%.4f' %
                  (epoch, loss.item() / y_pred.shape[0], acc(y, y_pred), nmi(y, y_pred)))




KeyboardInterrupt: 

In [ ]:
if __name__ == "__main__":
    import argparse
    import warnings

    parser = argparse.ArgumentParser(description='DSCNet')
    parser.add_argument('--db', default='coil20',
                        choices=['coil20', 'coil100', 'orl', 'reuters10k', 'stl'])
    parser.add_argument('--show-freq', default=10, type=int)
    parser.add_argument('--ae-weights', default=None)
    parser.add_argument('--save-dir', default='results')
    args = parser.parse_args()
    print(args)
    import os

    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    db = args.db
    if db == 'coil20':
        # load data
        data = sio.loadmat('datasets/COIL20.mat')
        x, y = data['fea'].reshape((-1, 1, 32, 32)), data['gnd']
        y = np.squeeze(y - 1)  # y in [0, 1, ..., K-1]

        # network and optimization parameters
        num_sample = x.shape[0]
        channels = [1, 15]
        kernels = [3]
        epochs = 40
        weight_coef = 1.0
        weight_selfExp = 75

        # post clustering parameters
        alpha = 0.04  # threshold of C
        dim_subspace = 12  # dimension of each subspace
        ro = 8  #
        warnings.warn("You can uncomment line#64 in post_clustering.py to get better result for this dataset!")
    elif db == 'coil100':
        # load data
        data = sio.loadmat('datasets/COIL100.mat')
        x, y = data['fea'].reshape((-1, 1, 32, 32)), data['gnd']
        y = np.squeeze(y - 1)  # y in [0, 1, ..., K-1]

        # network and optimization parameters
        num_sample = x.shape[0]
        channels = [1, 50]
        kernels = [5]
        epochs = 120
        weight_coef = 1.0
        weight_selfExp = 15

        # post clustering parameters
        alpha = 0.04  # threshold of C
        dim_subspace = 12  # dimension of each subspace
        ro = 8  #
    elif db == 'orl':
        # load data
        data = sio.loadmat('datasets/ORL_32x32.mat')
        x, y = data['fea'].reshape((-1, 1, 32, 32)), data['gnd']
        y = np.squeeze(y - 1)  # y in [0, 1, ..., K-1]
        # network and optimization parameters
        num_sample = x.shape[0]
        channels = [1, 3, 3, 5]
        kernels = [3, 3, 3]
        epochs = 700
        weight_coef = 2.0
        weight_selfExp = 0.2

        # post clustering parameters
        alpha = 0.2  # threshold of C
        dim_subspace = 3  # dimension of each subspace
        ro = 1  #

    dscnet = DSCNet(num_sample=num_sample, channels=channels, kernels=kernels)
    dscnet.to(device)

    # load the pretrained weights which are provided by the original author in
    # https://github.com/panji1990/Deep-subspace-clustering-networks
    ae_state_dict = torch.load('pretrained_weights_original/%s.pkl' % db)
    dscnet.ae.load_state_dict(ae_state_dict)
    print("Pretrained ae weights are loaded successfully.")

    train(dscnet, x, y, epochs, weight_coef=weight_coef, weight_selfExp=weight_selfExp,
          alpha=alpha, dim_subspace=dim_subspace, ro=ro, show=args.show_freq, device=device)
    torch.save(dscnet.state_dict(), args.save_dir + '/%s-model.ckp' % args.db)


最近的成果整理：
如上：复现了一个基于pytorch的深度子空间聚类网络模型
能够在人家的coil数据集上跑成功
但是在自己的indian_pines却无法复现
1，数据的维度和组织格式不同
2，代码报错信息显示维度不匹配，